In [35]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [36]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-02 01:18:19--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.3’

postgresql-42.2.16. 100%[===================>] 979.38K  5.60MB/s    in 0.2s    

2021-08-02 01:18:20 (5.60 MB/s) - ‘postgresql-42.2.16.jar.3’ saved [1002883/1002883]



In [37]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [38]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url_list ="https://ucb-data-group-project-zillow-s3.s3.us-west-1.amazonaws.com/final_list_price_data.csv"
spark.sparkContext.addFile(url_list)
list_data_df = spark.read.csv(SparkFiles.get("final_list_price_data.csv"), sep=",", header=True, inferSchema=True)

In [39]:
# Show DataFrame
list_data_df.show()

+--------------+---------+---------+--------------------+-----------+----------+----------+--------+
|   region_date|region_id|size_rank|         region_name|region_type|state_name|      date|   price|
+--------------+---------+---------+--------------------+-----------+----------+----------+--------+
|10200120171104|   102001|        0|       United States|    Country|      null|2017-11-04|275448.0|
|39491320171104|   394913|        1|        New York, NY|        Msa|        NY|2017-11-04|526850.0|
|75389920171104|   753899|        2|Los Angeles-Long ...|        Msa|        CA|2017-11-04|812252.0|
|39446320171104|   394463|        3|         Chicago, IL|        Msa|        IL|2017-11-04|319556.0|
|39451420171104|   394514|        4|Dallas-Fort Worth...|        Msa|        TX|2017-11-04|347537.0|
|39497420171104|   394974|        5|    Philadelphia, PA|        Msa|        PA|2017-11-04|263365.0|
|39469220171104|   394692|        6|         Houston, TX|        Msa|        TX|2017-11-04|

In [40]:
url_sale ="https://ucb-data-group-project-zillow-s3.s3.us-west-1.amazonaws.com/final_sale_price_data.csv"
spark.sparkContext.addFile(url_sale)
sale_data_df = spark.read.csv(SparkFiles.get("final_sale_price_data.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
sale_data_df.show()

+--------------+---------+---------+--------------------+-----------+----------+----------+--------+
|   region_date|region_id|size_rank|         region_name|region_type|state_name|      date|   price|
+--------------+---------+---------+--------------------+-----------+----------+----------+--------+
|10200120080223|   102001|        0|       United States|    Country|      null|2008-02-23|191138.0|
|39491320080223|   394913|        1|        New York, NY|        Msa|        NY|2008-02-23|    null|
|75389920080223|   753899|        2|Los Angeles-Long ...|        Msa|        CA|2008-02-23|516750.0|
|39446320080223|   394463|        3|         Chicago, IL|        Msa|        IL|2008-02-23|247988.0|
|39451420080223|   394514|        4|Dallas-Fort Worth...|        Msa|        TX|2008-02-23|143466.0|
|39497420080223|   394974|        5|    Philadelphia, PA|        Msa|        PA|2008-02-23|217527.0|
|39469220080223|   394692|        6|         Houston, TX|        Msa|        TX|2008-02-23|

In [41]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://ucb-data-group-project-zillow.csaw135fqqkl.us-west-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [44]:
# Write DataFrame to active_user table in RDS
list_data_df.write.jdbc(url=jdbc_url, table='list', mode=mode, properties=config)

In [45]:
# Write DataFrame to active_user table in RDS
sale_data_df.write.jdbc(url=jdbc_url, table='sale', mode=mode, properties=config)